# Intermediate SQL

## CASE statements
Contains a WHEN, THEN, and ELSE statement, finished with END
```SQL
 CASE WHEN x = 1 THEN "a" 
     WHEN x = 2 THEN "b"
     ELSE "c" END AS new_column
```

### Basic CASE statements
The European Soccer Database contains data about matches from 11 European countries from the 2011-2015 match seasons. In total, you will be working with data from 12,800 matches!

You'll notice that in the match table, there are two columns called `hometeam_id` and `awayteam_id`. Both ID's can join to the teamapiid column in the team table, but you cannot perform a join on both columns at the same time.

You can use a CASE statement to identify which teams played in a match. However, your first step is to determine which IDs correspond to the FC Schalke 04 and FC Bayern Munich teams by querying the team table. You will then use these IDs to examine matches played by these teams in the match table.

**match**

`
id	country_id	season	stage	date	hometeam_id	awayteam_id	home_goal	away_goal
757 	 1	     2011/2012	 1	2011-07-29	1773	     8635	       2	          1
758	  1	     2011/2012	 1	2011-07-30	9998	     9985	       1	          1
759	  1    	 2011/2012	 1	2011-07-30	9987	     9993	       3	          1`


**team**

`
id	team_api_id	team_long_name	team_short_name
1	  9987	        KRC Genk	         GEN
2	  9993	        Beerschot AC	     BAC
3	  10000	       SV Zulte-Waregem	 ZUL
`

   **Instructions**
   
**1.1**- Create a query that gives you the team IDs for FC Schalke 04 and FC Bayern Munich.

```SQL
SELECT
	-- Select the team long name and team API id
	team_long_name, team_api_id
	
FROM team 
-- Only include FC Schalke 04 and FC Bayern Munich
WHERE team_long_name IN ('FC Schalke 04', 'FC Bayern Munich');

```
`#OUTPUT
team_long_name	      team_api_id
FC Bayern Munich	     9823
FC Schalke 04	        10189`





**2.1** Create a CASE statement that identifies whether a match played in Germany included Bayern Munich, Schalke 04, or neither.<br>
**2.2** .Count the id column to determine the number of matches played by each team.<br>
**2.3** Group the query by the CASE statement.


```SQL
-- Identify if a match included Bayern Munich, Schalke 04, or neither
SELECT 
	CASE WHEN hometeam_id = 10189 OR awayteam_id = 10189 THEN 'FC Schalke 04'
        WHEN  hometeam_id = 9823  OR awayteam_id = 9823 THEN 'FC Bayern Munich'
         ELSE 'Other' END AS team_name,
	COUNT(id) AS matches
FROM match
WHERE country_id = 7809
-- Group by the CASE statement
GROUP BY  
	CASE WHEN hometeam_id = 10189 OR awayteam_id = 10189 THEN 'FC Schalke 04'
        WHEN  hometeam_id = 9823  OR awayteam_id = 9823 THEN 'FC Bayern Munich'
         ELSE 'Other' END
```
`OUTPUT
team_name	          matches
Other	                960
FC Schalke 04	        136
FC Bayern Munich     	128`


### CASE statements comparing column values

Querying a list of all matches in the 2011/2012 season where Barcelona was the home team. You will do this using a CASE statement that compares the values of two columns to create a new group -- wins, losses, and ties. The match table contains two columns -- home team ID, and away team ID, to join with the team table. Since you can only join team to one column at a time, you will create two queries in this and the next exercise to examine Barcelona's performance.

```SQL
SELECT m.date, t.team_long_name AS opponent
    CASE WHEN m.home_goal >  m.away_goal THEN 'Barcelone Win'
         WHEN m.home_goal < m.away_goal THEN 'Barcelona Loss :('
         ELSE 'Tie' END AS outcome

FROM match AS m
LEFT JOIN team AS t
ON m.awayteam_id = t.team_api_id
-- Filter for 2012/2013 and Barcelona as the home team
WHERE m.season = '2011/2012' AND m.hometeam_id = 8634
-- Order by the date, with most recent first
ORDER BY m.date ASC; 

```
`OUTPUT
2012-04-10	Getafe CF	 Barcelona win!
2012-04-21	Real Madrid   CFBarcelona loss :(
2012-05-02	Málaga CF	 Barcelona win!
2012-05-05	RCD           Espanyol	`




```SQL
-- Select matches where Barcelona was the away team
SELECT  
	m.date,
	t.team_long_name AS opponent,
	CASE WHEN m.home_goal < m.away_goal THEN 'Barcelona win!'
         WHEN m.home_goal > m.away_goal THEN 'Barcelona loss :(' 
         ELSE 'Tie' END AS outcome
FROM match AS m
-- Join the team table to match
LEFT JOIN team AS t ON m.hometeam_id = t.team_api_id
WHERE  m.season = '2011/2012' AND m.awayteam_id = 8634
ORDER BY m.date;
```
`OUTPUT
2011-09-10	Real Sociedad	             Tie
2011-09-21	Valencia CF	               Tie
2011-10-02	Real Sporting de Gijón	    Barcelona win!`

### CASE WHEN things to look out for
Barcelona and Real Madrid have been rival teams for more than 80 years. Matches between these two teams are given the name El Clásico (The Classic). In this exercise, you will query a list of matches played between these two rivals.


```SQL
SELECT 
	date,
	CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END as home,
	CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END as away,
	-- Identify all 5 possible match outcomes
	CASE WHEN home_goal > away_goal AND hometeam_id = 8634 THEN 'Barcelona win!'
          WHEN home_goal > away_goal AND hometeam_id = 8633 THEN 'Real Madrid win!'
          WHEN home_goal < away_goal AND awayteam_id = 8634 THEN 'Barcelona win!'
          WHEN home_goal < away_goal AND awayteam_id = 8633 THEN 'Real Madrid win!'
          ELSE 'Tie!' END AS outcome
FROM match
WHERE (awayteam_id = 8634 OR hometeam_id = 8634)
      AND (awayteam_id = 8633 OR hometeam_id = 8633);
```
`OUTPUT
date	      home	           away	          outcome
2011-12-10	Real Madrid CF	FC Barcelona	  Barcelona win!
2012-04-21	FC Barcelona	  Real Madrid CF	Real Madrid win!
2013-03-02	Real Madrid CF	FC Barcelona	  Real Madrid win!
2012-10-07	FC Barcelona	  Real Madrid CF	Tie!`